# DropGNN + Contrastive Learning Prototyping

Work based off of DropGNN

https://arxiv.org/pdf/2111.06283.pdf 

https://github.com/KarolisMart/DropGNN 

In [2]:
import os.path as osp
import numpy as np
import networkx as nx
import time
import random
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold


# Torch Geometric 
try: 
    from torch_geometric.data import DataLoader, Data
    from torch_geometric.data.dataloader import Collater
    from torch_geometric.datasets import TUDataset
    from torch_geometric.utils import degree
    from torch_geometric.utils.convert import from_networkx
    from torch_geometric.nn import GINConv, GINEConv, global_add_pool
except ModuleNotFoundError: 
    !pip install torch_geometric
    from torch_geometric.data import DataLoader, Data
    from torch_geometric.data.dataloader import Collater
    from torch_geometric.datasets import TUDataset
    from torch_geometric.utils import degree
    from torch_geometric.utils.convert import from_networkx
    from torch_geometric.nn import GINConv, GINEConv, global_add_pool
    
# Pytorch Metric Learning
try: 
    from pytorch_metric_learning import losses
except ModuleNotFoundError:
    !pip install pytorch-metric-learning
    from pytorch_metric_learning import losses

## Import Dataset

Use the IMDB-Binary dataset

In [3]:
class MyFilter(object):
    def __call__(self, data):
        return data.num_nodes <= 70

class MyPreTransform(object):
    def __call__(self, data):
        data.x = degree(data.edge_index[0], data.num_nodes, dtype=torch.long)
        data.x = F.one_hot(data.x, num_classes=69).to(torch.float)
        return data 

In [4]:
# Download data 
path = osp.join(osp.dirname(osp.realpath("./")), 'data', f'IMDB-BINARY')

dataset = TUDataset(
    path, 
    name = "IMDB-BINARY", 
    pre_transform = MyPreTransform(), 
    pre_filter = MyFilter()
)

In [5]:
print(dataset)

IMDB-BINARY(996)


## Original DropGIN Model

DropGIN model found in DropGNN paper modified to work without CLI arguments

In [6]:
class DropGIN(nn.Module):
    def __init__(self, hidden_units, dropout, use_aux_loss=True, num_runs=20, p=0.1):
        super(DropGIN, self).__init__()

        # Set starting parameters for model 
        num_features = dataset.num_features # Number of initial features 
        dim = hidden_units # Number of hidden units per layer 
        self.dropout = dropout # Dropout rate 
        self.use_aux_loss = use_aux_loss
        self.num_runs = num_runs
        self.p = p

        # Number of layers in model
        self.num_layers = 4

        self.convs = nn.ModuleList() # Made of num_layers GINConv (linear -> batchnorm1d -> relu -> linear)
        self.bns = nn.ModuleList() # Made of num_layers BatchNorm1d 
        self.fcs = nn.ModuleList() # Made of num_layers + 1 Linear layers mapping from num_features or dim to num_classes

        # Add initial layer from num_features to dim 
        self.convs.append(GINConv(nn.Sequential(nn.Linear(num_features, dim), nn.BatchNorm1d(dim), nn.ReLU(), nn.Linear(dim, dim))))
        self.bns.append(nn.BatchNorm1d(dim))
        self.fcs.append(nn.Linear(num_features, dataset.num_classes))
        self.fcs.append(nn.Linear(dim, dataset.num_classes))

        # Add additional layers from dim to dim 
        for i in range(self.num_layers-1):
            self.convs.append(GINConv(nn.Sequential(nn.Linear(dim, dim), nn.BatchNorm1d(dim), nn.ReLU(), nn.Linear(dim, dim))))
            self.bns.append(nn.BatchNorm1d(dim))
            self.fcs.append(nn.Linear(dim, dataset.num_classes))

        # Use aux_loss for dropGNN: made of num_layers + 1 linear layers 
        # Adds new module list of linear layers 
        if self.use_aux_loss:
            self.aux_fcs = nn.ModuleList()
            self.aux_fcs.append(nn.Linear(num_features, dataset.num_classes))
            for i in range(self.num_layers):
                self.aux_fcs.append(nn.Linear(dim, dataset.num_classes))
        
    def reset_parameters(self):
        # Resets parameters for Linear, GINConv, and BatchNorm1d layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.reset_parameters()
            elif isinstance(m, GINConv):
                m.reset_parameters()
            elif isinstance(m, nn.BatchNorm1d):
                m.reset_parameters()

    def forward(self, data, temp):
        # Note: num_runs in DropGNN is average number of nodes in each graph in dataset
        # Note: p is 2 * 1 / (1 + gamma), but for this project, p is selected to create augmented views 

        x = data.x
        edge_index = data.edge_index
        batch = data.batch
            
        # Do runs in parallel, by repeating the graphs in the batch
        x = x.unsqueeze(0).expand(self.num_runs, -1, -1).clone()   # Flattens features and creates num_runs copy of them 
        drop = torch.bernoulli(torch.ones([x.size(0), x.size(1)], device=x.device) * self.p).bool()   #  Returns a tensor of randomly dropped nodes based on p (p = probability of dropping) 
        x[drop] = torch.zeros([drop.sum().long().item(), x.size(-1)], device=x.device)  # Drop nodes from data  
        del drop

        # Run augmented subgraph through model 
        outs = [x]  # Used to store view of x after each layer 
        x = x.view(-1, x.size(-1))  # Swap dimensions of data features 
        run_edge_index = edge_index.repeat(1, self.num_runs) + torch.arange(self.num_runs, device=edge_index.device).repeat_interleave(edge_index.size(1)) * (edge_index.max() + 1) # Expand edge_index and augment values
        for i in range(self.num_layers):
            x = self.convs[i](x, run_edge_index)  # Run node features and edge indices through CONV layer 
            x = self.bns[i](x)  # Run resulting values through BatchNorm1d
            x = F.relu(x)   # Run final values through RELU
            outs.append(x.view(self.num_runs, -1, x.size(-1)))    # Rearrange dimensions and append to outs 
        del run_edge_index

        # TODO: Replace this with replacable head for contrastive learning and making predictions 
        # Aggregates results of runs by summing mean and applying random dropout (not dropping out nodes)
        out = None
        for i, x in enumerate(outs):
            x = x.mean(dim=0)
            x = global_add_pool(x, batch)
            x = F.dropout(self.fcs[i](x), p=self.dropout, training=self.training)
            if out is None:
                out = x
            else:
                out += x

        if self.use_aux_loss:
            aux_out = torch.zeros(self.num_runs, out.size(0), out.size(1), device=out.device)
            run_batch = batch.repeat(self.num_runs) + torch.arange(self.num_runs, device=edge_index.device).repeat_interleave(batch.size(0)) * (batch.max() + 1)
            for i, x in enumerate(outs):
                x = x.view(-1, x.size(-1))
                x = global_add_pool(x, run_batch)
                x = x.view(self.num_runs, -1, x.size(-1))
                x = F.dropout(self.aux_fcs[i](x), p=self.dropout, training=self.training)
                aux_out += x

            # Returns probabilities of each class based on aggregated results 
            return F.log_softmax(out, dim=-1), F.log_softmax(aux_out, dim=-1)
        else:
            return F.log_softmax(out, dim=-1), 0

## DropGNN + Contrastive Learning Model

Copy of DropGIN model with Contrastive Learning Forward methods added

Changes made: 
- Added more arguments when initializing model instead of depending on global variables 
- Changed how the forward function is called and run
- Add two different forward functions
    - Contrastive Forward: Runs graph data through the base model and returns a representation vector 
    - Prediction Forward: Runs graph data through the base model and returns a probablity vector for each class 

In [7]:
class DropGNN_Contrastive(nn.Module):
    def __init__(self, num_features, num_reps, num_classes, hidden_units, use_aux_loss=True):
        super(DropGIN, self).__init__()

        # Set starting parameters for model 
        self.num_features = num_features   # Number of initial features 
        self.num_reps = num_reps           # Number of features in representation vector 
        self.num_classes = num_classes     # Number of different classes
        self.dim = hidden_units            # Number of units for hidden layers
        self.use_aux_loss = use_aux_loss   # Whether to include aux loss to total loss

        # Number of layers in model
        self.num_layers = 4

        self.convs = nn.ModuleList()                     # Made of num_layers GINConv (linear -> batchnorm1d -> relu -> linear)
        self.bns = nn.ModuleList()                       # Made of num_layers BatchNorm1d 
        self.reps = nn.Linear(self.dim, self.num_reps)   # Layer between base model and contrastive learning representation
        self.fcs = nn.ModuleList()                       # Made of num_layers + 1 Linear layers mapping from num_features or dim to num_reps

        # Add initial layer from num_features to dim 
        self.convs.append(GINConv(nn.Sequential(nn.Linear(self.num_features, self.dim), nn.BatchNorm1d(self.dim), nn.ReLU(), nn.Linear(self.dim, self.dim))))
        self.bns.append(nn.BatchNorm1d(self.dim))
        self.fcs.append(nn.Linear(self.num_features, self.num_classes))
        self.fcs.append(nn.Linear(self.dim, self.num_classes))

        # Add additional layers from dim to dim 
        for i in range(self.num_layers-1):
            self.convs.append(GINConv(nn.Sequential(nn.Linear(self.dim, self.dim), nn.BatchNorm1d(self.dim), nn.ReLU(), nn.Linear(self.dim, self.dim))))
            self.bns.append(nn.BatchNorm1d(self.dim))
            self.fcs.append(nn.Linear(self.dim, self.num_classes))

        # Use aux_loss for dropGNN: made of num_layers + 1 linear layers 
        # Adds new module list of linear layers 
        if self.use_aux_loss:
            self.aux_fcs = nn.ModuleList()
            self.aux_fcs.append(nn.Linear(self.num_features, self.num_classes))
            for i in range(self.num_layers):
                self.aux_fcs.append(nn.Linear(self.dim, self.num_classes))
        
    def reset_parameters(self):
        # Resets parameters for Linear, GINConv, and BatchNorm1d layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.reset_parameters()
            elif isinstance(m, GINConv):
                m.reset_parameters()
            elif isinstance(m, nn.BatchNorm1d):
                m.reset_parameters()
                
    def forward(self, data, mode="test", p=None, dropout=None, num_runs=20):
        # Runs different modes based on whether running contrastive loss or making predictions
        if mode == 'contrastive':
            return self.contrastive(data, p, num_runs)
        else:
            return self.prediction(data, dropout, num_runs)
        
    def contrastive(self, data, p, num_runs):
        # Trains contrastive model and representation vector model 
        
        # Note: num_runs in DropGNN is average number of nodes in each graph in dataset
        # Note: p is 2 * 1 / (1 + gamma), but for this project, p is selected to create augmented views 
        
        self.p = p
        self.num_runs = num_runs
        
        x = data.x
        edge_index = data.edge_index
        batch = data.batch 
        
        # Do runs in parallel, by repeating the graphs in the batch
        x = x.unsqueeze(0).expand(self.num_runs, -1, -1).clone()   # Flattens features and creates num_runs copy of them 
        drop = torch.bernoulli(torch.ones([x.size(0), x.size(1)], device=x.device) * self.p).bool()   #  Returns a tensor of randomly dropped nodes based on p (p = probability of dropping) 
        x[drop] = torch.zeros([drop.sum().long().item(), x.size(-1)], device=x.device)  # Drop nodes from data  
        del drop
        
        # Allow gradients to update base model 
        if self.training:
            for layer in self.convs:
                for p in layer.parameters():
                    p.requires_grad = True

            for layer in self.bns:
                for p in layer.parameters():
                    p.requires_grad = True
        
        # Run augmented subgraph through model 
        outs = [x]  # Used to store view of x after each layer 
        x = x.view(-1, x.size(-1))  # Swap dimensions of data features 
        run_edge_index = edge_index.repeat(1, self.num_runs) + torch.arange(self.num_runs, device=edge_index.device).repeat_interleave(edge_index.size(1)) * (edge_index.max() + 1) # Expand edge_index and augment values
        for i in range(self.num_layers):
            x = self.convs[i](x, run_edge_index)  # Run node features and edge indices through CONV layer 
            x = self.bns[i](x)  # Run resulting values through BatchNorm1d
            x = F.relu(x)   # Run final values through RELU
            outs.append(x.view(self.num_runs, -1, x.size(-1)))    # Rearrange dimensions and append to outs 
        del run_edge_index
        
        # Turn intermediate representation into contrastive representation
        x = self.reps(x)
        return x
    
    def prediction(self, data, dropout, num_runs):
        self.dropout = dropout
        self.num_runs = num_runs
        
        # Create intermediate representations 
        x = data.x 
        edge_index = data.edge_index
        batch = data.batch 
        
        # Do runs in parallel, by repeating the graphs in the batch
        x = x.unsqueeze(0).expand(self.num_runs, -1, -1).clone()   # Flattens features and creates num_runs copy of them 
        drop = torch.bernoulli(torch.ones([x.size(0), x.size(1)], device=x.device) * self.p).bool()   #  Returns a tensor of randomly dropped nodes based on p (p = probability of dropping) 
        x[drop] = torch.zeros([drop.sum().long().item(), x.size(-1)], device=x.device)  # Drop nodes from data  
        del drop
        
        # Stop gradients from updating base model 
        for layer in self.convs:
            for p in layer.parameters():
                p.requires_grad = False
                
        for layer in self.bns:
            for p in layer.parameters():
                p.requires_grad = False
        
        # Run augmented subgraph through model 
        outs = [x]  # Used to store view of x after each layer 
        x = x.view(-1, x.size(-1))  # Swap dimensions of data features 
        run_edge_index = edge_index.repeat(1, self.num_runs) + torch.arange(self.num_runs, device=edge_index.device).repeat_interleave(edge_index.size(1)) * (edge_index.max() + 1) # Expand edge_index and augment values
        for i in range(self.num_layers):
            x = self.convs[i](x, run_edge_index)  # Run node features and edge indices through CONV layer 
            x = self.bns[i](x)  # Run resulting values through BatchNorm1d
            x = F.relu(x)   # Run final values through RELU
            outs.append(x.view(self.num_runs, -1, x.size(-1)))    # Rearrange dimensions and append to outs 
        del run_edge_index
        
        # Aggregates results of runs by summing mean and applying random dropout (not dropping out nodes)
        out = None
        for i, x in enumerate(outs):
            x = x.mean(dim=0)
            x = global_add_pool(x, batch)
            x = F.dropout(self.fcs[i](x), p=self.dropout, training=self.training)
            if out is None:
                out = x
            else:
                out += x

        if self.use_aux_loss:
            aux_out = torch.zeros(self.num_runs, out.size(0), out.size(1), device=out.device)
            run_batch = batch.repeat(self.num_runs) + torch.arange(self.num_runs, device=edge_index.device).repeat_interleave(batch.size(0)) * (batch.max() + 1)
            for i, x in enumerate(outs):
                x = x.view(-1, x.size(-1))
                x = global_add_pool(x, run_batch)
                x = x.view(self.num_runs, -1, x.size(-1))
                x = F.dropout(self.aux_fcs[i](x), p=self.dropout, training=self.training)
                aux_out += x

            # Returns probabilities of each class based on aggregated results 
            return F.log_softmax(out, dim=-1), F.log_softmax(aux_out, dim=-1)
        else:
            return F.log_softmax(out, dim=-1), 0

## Training Modules

NOTE: Currently, all training modules are for base DropGNN model

Changes Made: 
- Added training function and validation function for contrastive learning 
- Added training function, validation function, and testing function for prediction task

### DropGIN Training Modules

In [42]:
# Default training function for original DropGIN
def train(model, epoch, loader, optimizer):
    # Set model to training 
    model.train()

    # Run data through model and update model 
    loss_all = 0
    n = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        logs, aux_logs = model(data) 
        loss = F.nll_loss(logs, data.y) 
    
        if model.use_aux_loss:
            aux_loss = F.nll_loss(aux_logs.view(-1, aux_logs.size(-1)), data.y.unsqueeze(0).expand(aux_logs.size(0), -1).clone().view(-1))
            loss = 0.75 * loss + 0.25 * aux_loss 
    
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        n += len(data.y)
        optimizer.step()

    return loss_all / n

In [31]:
# Default validation function for original DropGIN, uses nll loss 
def validate(model, loader):
    model.eval()
    with torch.no_grad():
        loss_all = 0
        for data in loader:
            data = data.to(device)
            logs, aux_logs = model(data)
            loss_all += F.nll_loss(logs, data.y, reduction="sum").item()
    return loss_all / len(loader.dataset)

In [43]:
# Default test function for original DropGIN, uses accuracy 
def test(model, loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        for data in loader:
            data = data.to(device)
            logs, aux_logs = model(data) 
            pred = logs.max(1)[1]
            correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

### Contrastive Learning Training Modules

In [9]:
# Used to train contrastive model 
def train_contrastive(model, loader, optimizer, p = 0.1):
    # Set model to training
    model.train()
    
    # Create self-supervised loss function
    loss_func = losses.SelfSupervisedLoss(losses.TripletMarginLoss())
    
    # Run data through model and update model
    loss_all = 0
    n = 0 
    for data in loader: 
        data = data.to(device)
        optimizer.zero_grad()
        embeddings = model(data, mode = "contrastive", p = 0.0)
        augmented = model(data, mode = "contrastive", p = p)
        loss = loss_func(embeddings, augmented)
        loss.backward()
        optimizer.step() 
        
        loss_all += data.num_graphs * loss.item()
        n += data.num_graphs
    return loss_all / n

In [ ]:
# Used to validate contrastive model 
def valid_contrastive(model, loader, p = 0.1):
    # Set model to eval
    model.eval()
    
    with torch.no_grad():
        loss_all = 0
        for data in loader: 
            data = data.to(device)
            embeddings = model(data, mode = "contrastive", p = 0.0)
            augmented = model(data, mode = "contrastive", p = p)
            loss = loss_func(embeddings, augmented)
            
            loss_all += data.num_graphs * loss.item()
            n += data.num_graphs
            
    return loss_all / n

### Prediction Training Modules

In [ ]:
# Used to train prediction model AFTER contrastive learning 
def train_prediction(model, loader, optimizer, dropout = 0.5):
    # Set model to training
    model.train()
    
    # Run data through model and update model 
    loss_all = 0
    n = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        logs, aux_logs = model(data, mode = "prediction", dropout = dropout)
        loss = F.nll_loss(logs, data.y)
        
       if model.use_aux_loss:
            aux_loss = F.nll_loss(aux_logs.view(-1, aux_logs.size(-1)), data.y.unsqueeze(0).expand(aux_logs.size(0), -1).clone().view(-1))
            loss = 0.75 * loss + 0.25 * aux_loss 
    
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        n += len(data.y)
        optimizer.step()

    return loss_all / n

In [ ]:
# Used to validate prediction model AFTER contrastive learning, returns loss 
def valid_prediction(model, loader, dropout = 0.5):
    # Set model to eval
    model.eval()
    
    # Run data through model
    with torch.no_grad():
        loss_all = 0
        n = 0
        for data in loader:
            data = data.to(device)
            logs, aux_logs = model(data, mode = "prediction", dropout = dropout)
            loss = F.nll_loss(logs, data.y)

            if model.use_aux_loss:
                aux_loss = F.nll_loss(aux_logs.view(-1, aux_logs.size(-1)), data.y.unsqueeze(0).expand(aux_logs.size(0), -1).clone().view(-1))
                loss = 0.75 * loss + 0.25 * aux_loss 
                
            loss_all += data.num_graphs * loss.item()
            n += len(data.y)

    return loss_all / n

In [ ]:
# Used to test prediction model AFTER contrastive learning, returns accuracy
def test_prediction(model, loader, dropout = 0.5):
    # Set model to eval
    model.eval() 
    
    # Run data through model 
    with torch.no_grad():
        correct = 0
        for data in loader: 
            data = data.to(device)
            logs, aux_logs = model(data, mode = "prediction", dropout = dropout)
            pred = logs.max(1)[1]
            correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

## Split Dataset into K-Folds

In [33]:
def separate_data(dataset_len, seed=0):
    folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    idx_list = []
    for idx in folds.split(np.zeros(dataset_len), np.zeros(dataset_len)):
        idx_list.append(idx)
    return idx_list

## Run DropGIN Model

In [34]:
# Set random seeds
torch.manual_seed(0)
np.random.seed(0)

# Set batch size
BATCH = 32    # Default batch size in DropGNN

# Set number of epochs
NUM_EPOCHS = 100

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [38]:
# Create model
model = DropGIN(64, 0.5).to(device)

# Split dataset
n = len(dataset)
splits = separate_data(n, seed=0)

In [ ]:
# Train model
acc = []
for i, (train_idx, valid_idx) in enumerate(splits[2:3]): # TODO: Change back to all splits
    model.reset_parameters()    # Resets upon every new fold 
    lr = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5) # Used to learn learning rate 
  
    valid_dataset = dataset[valid_idx.tolist()]
    train_dataset = dataset[train_idx.tolist()]

    valid_loader = DataLoader(valid_dataset, batch_size=BATCH)
    # train_loader = DataLoader(train_dataset, batch_size=BATCH)
    train_loader = torch.utils.data.DataLoader(train_dataset, sampler=torch.utils.data.RandomSampler(train_dataset, replacement=True, num_samples=int(len(train_dataset)*50/(len(train_dataset)/BATCH))), batch_size=BATCH, drop_last=False, collate_fn=Collater(follow_batch=[],exclude_keys=[]))

    # Train model for multiple epochs
    valid_acc = 0 
    acc_temp = []
    for epoch in range(NUM_EPOCHS):
        if epoch % 25 == 0:
            start = time.time()

        lr = scheduler.optimizer.param_groups[0]['lr']
        train_loss = train(model, epoch, train_loader, optimizer)
        scheduler.step()
        valid_acc = test(model, valid_loader)
        acc_temp.append(valid_acc)

        if epoch % 25 == 0:
            print('Epoch: {:03d}, LR: {:7f}, Train Loss: {:.7f}, '
                'Val Loss: {:.7f}, Test Acc: {:.7f}, Time: {:7f}'.format(
                    epoch, lr, train_loss, 0, valid_acc, time.time() - start), flush=True)

    acc.append(torch.tensor(acc_temp))

acc = torch.stack(acc, dim=0)
acc_mean = acc.mean(dim=0)
best_epoch = acc_mean.argmax().item()

In [ ]:
# Note: std won't work if only using one k-fold 

print('---------------- Final Epoch Result ----------------')
print('Mean: {:7f}, Std: {:7f}'.format(acc[:,-1].mean(), acc[:,-1].std()))
print(f'---------------- Best Epoch: {best_epoch} ----------------')
print('Mean: {:7f}, Std: {:7f}'.format(acc[:,best_epoch].mean(), acc[:,best_epoch].std()), flush=True)